# Exercice semaine 3

Perform the following tasks using the dataframe (containing all protected areas) that you created last week with the Vahatra data: 

1. Using statsmodels, fit an OLS model with the deforestation rate over the period 2000 - 2023 as your response variable and the year of creation of the protected area as your predictor variable. Design your matrix of predictors "manually" using the DataFrame method of pandas. Print a summary of the results and discuss the sign, magnitude and statistical significance of the coefficient on deforest_2000_2023. 
2. Train a similar OLS model using the ModelSpec method of the ISLP package to design your matrix of predictors.
3. Generate predictions for Corridor Forestier Ambositra-Vondrozo, Mantadia, Menabe Antimena, and Montagne d'Ambre/Forêt d'Ambre, using the OLS model that you just fitted. Create an array or a dataframe to generate a table which compares the predicted values and the true values of the deforestation rate for these four protected areas.
4. Compute confidence intervals and prediction intervals for the predicted deforestation rates of these four areas and print them.

In [1]:
pip install ISLP

ERROR: Could not find a version that satisfies the requirement ISLP (from versions: none)
ERROR: No matching distribution found for ISLP
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Install and load packages

import numpy as np #numerical operations
import pandas as pd #data wrangling

!pip install openpyxl #import Excel data
import openpyxl
 
import statsmodels.api as sm #regression models
from statsmodels.stats.outliers_influence \
     import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

from ISLP import load_data #matrix design and other regression related functions
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
#Data preparation
pas = pd.read_excel('Vahatra_defor_noNA.xlsx')
pas.columns = pas.columns.str.replace("-01-01_treecover_ha", "", regex=False)
pas['deforest_2000_2023'] = (pas['treecover_area_2000']-pas['treecover_area_2023'])/pas['treecover_area_2000']*100

**1. Using statsmodels, fit an OLS model with the deforestation rate over the period 2000 - 2023 as your response variable and the year of creation of the protected area as your predictor variable. Design your matrix of predictors "manually" using the DataFrame method of pandas. Print a summary of the results and discuss the sign, magnitude and statistical significance of the coefficient on deforest_2000_2023.**

In [18]:
#Design matrix of predictors

X = pd.DataFrame({'intercept': np.ones(pas.shape[0]),
                  'date_creation': pas['date_creation'].dt.year})
X

,intercept,date_creation
0,1.0,2015
1,1.0,2015
2,1.0,1958
3,1.0,1956
4,1.0,2015
...,...,...
92,1.0,2015
93,1.0,2015
94,1.0,2015
95,1.0,2015


In [15]:
#Create response vector
y = pd.DataFrame(pas['deforest_2000_2023'])
y

,deforest_2000_2023
0,21.022159
1,13.385643
2,14.314523
3,42.940478
4,24.057451
...,...
92,38.989696
93,7.987383
94,12.981940
95,5.552914


In [20]:
#Specify and fit the model
defor_model = sm.OLS(y, X) #Specifies the model 
defor_results = defor_model.fit() #Fit the model 
defor_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     deforest_2000_2023   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     4.296
Date:                Wed, 11 Feb 2026   Prob (F-statistic):             0.0409
Time:                        16:45:53   Log-Likelihood:                -398.58
No. Observations:                  97   AIC:                             801.2
Df Residuals:                      95   BIC:                             806.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
intercept      -259.2753    132.132     -1.962      0.053    -521.591       3.040
date_creation     0.1370      0.066      2.073      0.041       0.006       0.268
==============================================================================
Omnibus:                       33.103   Durbin-Watson:                   2.250
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.303
Skew:                           1.480   Prob(JB):                     9.80e-13
Kurtosis:                       5.217   Cond. No.                     1.75e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The deforestation rate during the period 2000-2023 is positively correlated with the year of creation of the protected areas. There is a 0.13 percentage point increase in the deforestation rate for every 1 year increase in the date of creation of a protected area. The predictor variable explains 4.3 percent of the variance in the response variable.

**2. Train a similar OLS model using the ModelSpec method of the ISLP package to design your matrix of predictors.**

In [23]:
# Alternative matrix design
pas['year_creation'] = pas['date_creation'].dt.year
design = MS(['year_creation'])
Z = design.fit_transform(pas)
Z

,intercept,year_creation
0,1.0,2015
1,1.0,2015
2,1.0,1958
3,1.0,1956
4,1.0,2015
...,...,...
92,1.0,2015
93,1.0,2015
94,1.0,2015
95,1.0,2015


In [24]:
#Specify and fit the model
defor_model2 = sm.OLS(y, Z) #Specifies the model 
defor_results2 = defor_model2.fit() #Fit the model 
defor_results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     deforest_2000_2023   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     4.296
Date:                Wed, 11 Feb 2026   Prob (F-statistic):             0.0409
Time:                        16:55:27   Log-Likelihood:                -398.58
No. Observations:                  97   AIC:                             801.2
Df Residuals:                      95   BIC:                             806.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
intercept      -259.2753    132.132     -1.962      0.053    -521.591       3.040
year_creation     0.1370      0.066      2.073      0.041       0.006       0.268
==============================================================================
Omnibus:                       33.103   Durbin-Watson:                   2.250
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.303
Skew:                           1.480   Prob(JB):                     9.80e-13
Kurtosis:                       5.217   Cond. No.                     1.75e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**3. Generate predictions for Corridor Forestier Ambositra-Vondrozo, Mantadia, Menabe Antimena, and Montagne d'Ambre/Forêt d'Ambre, using the OLS model that you just fitted. Create an array or a dataframe to generate a table which compares the predicted values and the true values of the deforestation rate for these four protected areas.**

In [52]:
#Filter using names
names = (pas['nom'].str.contains("Ambositra", na=False)) | (pas['nom'] == 'Mantadia') |\
(pas['nom'] == 'Menabe Antimena') | (pas['nom'] == "Montagne d'Ambre/Forêt d'Ambre")
pas_selected = pas.loc[names, :] 
pas_selected

,nom,cat_iucn,creation,date_creation,date_modification,mention_changement,hectares,num_atlas_,full_name,province,...,Groupe,surface_m2,surface_ha,surface_km2,couv_foret_2000,altitude,pente,dist_ville,deforest_2000_2023,year_creation
14,Corridor Forestier Ambositra-Vondrozo,V,Décret n°2015-755 du 28 avril 2015,2015-04-28,NaT,False,311881.773,45,Paysage Harmonieux Protégé du Corridor Foresti...,Fianarantsoa,...,Contrôle,3.127013e+09,312701.339855,3127.013399,97.174269,1033.648579,15.894881,199.063785,20.464798,2015
27,Menabe Antimena,V,Décret n°2015-762 du 28 avril 2015,2015-04-28,NaT,False,201927.135,85,Paysage Harmonieux Protégé de Menabe Antimena,Toliary,...,Contrôle,2.025998e+09,202599.773362,2025.997734,49.953114,24.978459,1.706078,96.167995,46.953912,2015
65,Mantadia,II,Changement de limite le 07 aout 2002,1989-01-11,2002-08-07,True,15455.784,40,Parc National de Mantadia,Toamasina,...,Traitement,1.550288e+08,15502.879962,155.028800,100.045036,1009.547713,14.700914,218.375635,10.296507,1989
73,Montagne d'Ambre/Forêt d'Ambre,II,"Créée le 28.10.58, changement limite; PN et RS...",1958-10-28,2015-04-28,True,30537.216,4,Parc National de Montagne d'Ambre,Antsiranana,...,Traitement,3.068160e+08,30681.595856,306.815959,91.346273,830.480900,12.686856,146.454545,1.816590,1958


In [40]:
#Matrix design
X_selected = design.transform(pas_selected)
X_selected

,intercept,year_creation
14,1.0,2015
27,1.0,2015
65,1.0,1989
73,1.0,1958


In [56]:
predict_selected = defor_results.get_prediction(X_selected) #compute the predictions
predict_mean = predict_selected.predicted_mean #see the results by extracting the mean 
predict_mean

array([16.74702799, 16.74702799, 13.18544984,  8.93895282])

In [57]:
predicted_df = pd.DataFrame({'true_deforest_2000_2023': pas_selected['deforest_2000_2023'],\
                             'pred_deforest_2000_2023':  predict_mean})
predicted_df

,true_deforest_2000_2023,pred_deforest_2000_2023
14,20.464798,16.747028
27,46.953912,16.747028
65,10.296507,13.185450
73,1.816590,8.938953


**4. Compute confidence intervals and prediction intervals for the predicted deforestation rates of these four areas and print them.**

In [44]:
predict_selected.conf_int(alpha=0.05) #produce 95% confidence intervals

array([[13.08780445, 20.40625153],
       [13.08780445, 20.40625153],
       [ 9.90809694, 16.46280275],
       [ 2.77417069, 15.10373495]])

In [45]:
predict_selected.conf_int(obs=True, alpha=0.05) #Prediction intervals

array([[-13.03366367,  46.52771966],
       [-13.03366367,  46.52771966],
       [-16.55073554,  42.92163523],
       [-21.25217532,  39.13008095]])